In [1]:
import pip

package_names = ['lightgbm', 'awswrangler', 'sagemaker'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

NumExpr defaulting to 4 threads.


### COnfig

In [3]:
from UTILITARIO_CODE.utils import targets

proyecto = 'propension'
normal = [1]
target = targets[7]
clasif = 'normal' if len(normal) == 1 else 'all'

now = datetime.now()
sess = sagemaker.session.Session()
s3 = boto3.resource('s3')
bucket = sess.default_bucket() 
region = boto3.Session().region_name

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-07-31 03:16:12.465215


### Seteado utilitario

In [4]:
BASE_DIR

'/home/ec2-user/SageMaker'

In [5]:
path_instancia_util = os.path.join(BASE_DIR, 'PROPENSION/UTILITARIO_CODE/utils.py')
path_instancia_util

'/home/ec2-user/SageMaker/PROPENSION/UTILITARIO_CODE/utils.py'

In [6]:
uri_code = 's3://{}/vpc/{}/code'.format(bucket, proyecto)
prefix_utils = '{}/utils.py'.format(uri_code.split(bucket)[-1][1:])
prefix_utils

'vpc/propension/code/utils.py'

In [7]:
s3.Bucket(bucket).upload_file(path_instancia_util, prefix_utils)

### RUtas

In [8]:
uri_output = 's3://{}/vpc/{}/output'.format(bucket, proyecto)
print('uri_output:', uri_output)

uri_train = '{}/train_{}_clasif_{}.csv'.format(uri_output, target.lower(), str(clasif))
uri_valid = '{}/validation{}_clasif_{}.csv'.format(uri_output, target.lower(), str(clasif))
uri_segui = '{}/seguimiento.csv'.format(uri_output)
#train_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
sufijo = '{}_clasif_{}'.format(target.lower(), clasif)

print("uri_train: ", uri_train)
print("uri_valid: ", uri_valid)
print("uri_segui: ", uri_segui)
print("sufijo: ", sufijo)

print("\n", "="*25, "\n")
uri_raw = 's3://{}/vpc/{}/raw'.format(bucket, proyecto)
print('uri_raw:', uri_raw)

uri_copy_train = '{}/{}/train.csv'.format(uri_raw, sufijo)
uri_copy_valid = '{}/{}/validation.csv'.format(uri_raw, sufijo)
uri_copy_segui = '{}/{}/seguimiento.csv'.format(uri_raw, sufijo)

print("uri_copy_train: ", uri_copy_train)
print("uri_copy_valid: ", uri_copy_valid)
print("uri_copy_segui: ", uri_copy_segui)

uri_output: s3://sagemaker-us-east-1-058528764918/vpc/propension/output
uri_train:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output/train_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
uri_valid:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output/validationtarget_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
uri_segui:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output/seguimiento.csv
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


uri_raw: s3://sagemaker-us-east-1-058528764918/vpc/propension/raw
uri_copy_train:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/train.csv
uri_copy_valid:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/validation.csv
uri_copy_segui:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal/seguimiento.csv


In [9]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_train.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_train.split(bucket)[-1][1:]
)

CPU times: user 1.23 s, sys: 45.5 ms, total: 1.28 s
Wall time: 21.8 s


In [10]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_valid.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_valid.split(bucket)[-1][1:]
)

CPU times: user 359 ms, sys: 8.12 ms, total: 367 ms
Wall time: 7.03 s


In [11]:
%%time
s3.meta.client.copy({
    'Bucket': bucket,
    'Key': uri_segui.split(bucket)[-1][1:]
},
    bucket,
    uri_copy_segui.split(bucket)[-1][1:]
)

CPU times: user 473 ms, sys: 25.1 ms, total: 498 ms
Wall time: 8.67 s


In [12]:
uri_input = '{}/{}'.format(uri_raw, sufijo)
uri_salida = uri_input.replace('raw', 'split')
print("uri_input: ", uri_input)
print("uri_salida: ", uri_salida)

print("\n", "="*25, "\n")

path_container_input = '/opt/ml/processing/input'
path_container_utils= path_container_input + '/utils'
path_container_output = path_container_input.replace('input', 'output')
print('path_container_input: ', path_container_input)
print('path_container_output: ', path_container_output)
print('path_container_utils: ', path_container_utils)

uri_input:  s3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal
uri_salida:  s3://sagemaker-us-east-1-058528764918/vpc/propension/split/target_desembolso_f2m_mayor_30_menor_180_clasif_normal


path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils


### Desacoplado

In [13]:
processing_tags = [{'Key': 'cost-center', 'Value': 'TF2WorkflowProcessing'}]

sklearn_processor_a = SKLearnProcessor(
    framework_version='0.23-1',
    role=get_execution_role(),
    instance_type='ml.m5.4xlarge',
    instance_count=1,
    tags=processing_tags
)

Same images used for training and inference. Defaulting to image scope: inference.
Defaulting to only available Python version: py3


In [14]:
%%time
processing_job_name = "propension-workflow-{}".format(strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

sklearn_processor_a.run(
    code='preparate_train_valid.py',  #'/opt/ml/processing/codigo_proceso/' , '/opt/ml/processing/input/code'
    job_name=processing_job_name,
    inputs=[
        ProcessingInput(
            source=uri_input,
            destination=path_container_input,
            s3_data_distribution_type='ShardedByS3Key'
        ),
        ProcessingInput(
            source=uri_code,
            destination=path_container_utils,
            s3_data_distribution_type='ShardedByS3Key'
        ),
    ],
    outputs=[
        ProcessingOutput(
            output_name='result',
            destination=uri_salida,
            source=path_container_output
        )
    ],
        arguments=["--sufijo", sufijo]
)

preprocessing_job_description = sklearn_processor_a.jobs[-1].describe()


Job Name:  propension-workflow-2021-07-31-03-16-50
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/raw/target_desembolso_f2m_mayor_30_menor_180_clasif_normal', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/vpc/propension/code', 'LocalPath': '/opt/ml/processing/input/utils', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'ShardedByS3Key', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-058528764918/propension-workflow-2021-07-31-03-16-50/input/code/preparate_train_valid.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistribut